In [2]:
# Test Pareto Optimizer

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs, Trial
from robyn.modeling.pareto.pareto_optimizer import ParetoOptimizer
from robyn.data.entities.enums import DependentVarType, PaidMediaSigns, OrganicSigns, ContextSigns

from utils.data_mapper import import_data, load_data_from_json


/Users/funny/Documents/git/Robyn/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data from JSON
loaded_data = load_data_from_json(
    "/Users/funny/Documents/git/Robyn/python/src/tutorials/resources/test_data.json"
    # "/Users/funny/Downloads/exported_data.json"
)
imported_data = import_data(loaded_data)
model_outputs = imported_data["model_outputs"]
display((model_outputs.hyper_bound_ng))


,facebook_S_alphas,facebook_S_gammas,facebook_S_thetas,newsletter_alphas,newsletter_gammas,newsletter_thetas,ooh_S_alphas,ooh_S_gammas,ooh_S_thetas,print_S_alphas,print_S_gammas,print_S_thetas,search_S_alphas,search_S_gammas,search_S_thetas,tv_S_alphas,tv_S_gammas,tv_S_thetas,train_size,lambda
0,0.5,0.3,0.0,0.5,0.3,0.1,0.5,0.3,0.1,0.5,0.3,0.1,0.5,0.3,0.0,0.5,0.3,0.3,0.5,0
1,3.0,1.0,0.3,3.0,1.0,0.4,3.0,1.0,0.4,3.0,1.0,0.4,3.0,1.0,0.3,3.0,1.0,0.8,0.8,1


In [4]:

mmm_data = imported_data["mmm_data"]
# display(mmm_data.data.head())
# Display Model Outputs

model_outputs = imported_data["model_outputs"]
# display((model_outputs.trials[0].result_hyp_param))

hyperparameters = imported_data["hyperparameters"]
# display(hyperparameters)

In [5]:

# 3. Create ParetoOptimizer instance
pareto_optimizer = ParetoOptimizer(mmm_data, model_outputs, hyperparameters)

# 4. Run optimize function
pareto_result = pareto_optimizer.optimize(pareto_fronts="auto", min_candidates=3)


>>> Calculating response curves for all models' media variables (10080)...


In [ ]:

# 5. Check results
print("Pareto Optimization Results:")
print(f"Number of Pareto solutions: {len(pareto_result.pareto_solutions)}")
print(f"Number of Pareto fronts: {pareto_result.pareto_fronts}")
print("\nPareto-optimal solutions:")
print(pareto_result.result_hyp_param[['solID', 'nrmse', 'decomp.rssd', 'mape', 'robynPareto']])

print("\nAggregated decomposition results:")
print(pareto_result.x_decomp_agg.head())

print("\nMedia vector collection:")
print(pareto_result.media_vec_collect.head())

print("\nDecomposition vector collection:")
print(pareto_result.x_decomp_vec_collect.head())

# 6. Validate logic
assert len(pareto_result.pareto_solutions) > 0, "No Pareto-optimal solutions found"
assert pareto_result.pareto_fronts > 0, "Invalid number of Pareto fronts"
assert not pareto_result.result_hyp_param.empty, "Empty result_hyp_param DataFrame"
assert not pareto_result.x_decomp_agg.empty, "Empty x_decomp_agg DataFrame"
assert not pareto_result.media_vec_collect.empty, "Empty media_vec_collect DataFrame"
assert not pareto_result.x_decomp_vec_collect.empty, "Empty x_decomp_vec_collect DataFrame"

print("\nAll assertions passed. The optimize function is working as expected.")